In [1]:
from __future__ import division, print_function
import os
import numpy as np
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import pickle
import operator

In [2]:
app = Flask(__name__)

dataset = []
def loadDataset(filename):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

loadDataset("my.dat")

In [3]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    
    #Method to calculate distance between two instances.
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    print("distane is",distance)
    return distance


In [4]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    print("neighbors is ",neighbors)
    return neighbors   

In [5]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    print("sorter is ",sorter)
    return sorter[0][0]

In [6]:
print('Model loaded. Check http://127.0.0.1:5000/')

Model loaded. Check http://127.0.0.1:5000/


In [7]:

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('music.html')

In [8]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']
        
        # Save the file to ./uploads
        basepath = "D:\TSB Projects\Music Genre Detection\Flask"
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        print(file_path)
        i=1
        
        results = {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 
                   6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'}
    
        (rate,sig)=wav.read(file_path)
        print(rate,sig)
        mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature=(mean_matrix,covariance,0)
        pred=nearestClass(getNeighbors(dataset ,feature , 8))
        
        print("predicted genre = ",pred,"class = ",results[pred])
        return "This song is classified as a "+str(results[pred])

In [ ]:

if __name__ == '__main__':
    app.run(threaded = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jun/2021 10:47:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2021 10:47:37] "GET /favicon.ico HTTP/1.1" 404 -


D:\TSB Projects\Music Genre Detection\Flask\uploads\pop.00008.wav
22050 [15446 13975 13481 ...  1187 -2550 -3964]
distane is 51.5756754683669
distane is 45.86621092550392
distane is 71.9122264208388
distane is 36.47512197152152
distane is 49.55434300106307
distane is 56.90976737942347
distane is 64.75539306142522
distane is 72.95262447998167
distane is 79.18317254141733
distane is 88.00844119471924
distane is 87.67124816688832
distane is 75.93691671135534
distane is 82.12809473926518
distane is 66.79583766093091
distane is 78.10947266731296
distane is 61.23790916640533
distane is 58.90221060292073
distane is 68.07935656670855
distane is 93.33387884285881
distane is 125.46766426968043
distane is 81.07077022669202
distane is 70.95260982400842
distane is 103.62255541586578
distane is 81.0060708492078
distane is 67.96090197312637
distane is 100.64143272794692
distane is 72.6453202286046
distane is 267.21050791813207
distane is 96.06627494339176
distane is 149.17886977451843
distane is 77.3

distane is 37.88780955817873
distane is 40.403267683659145
distane is 37.34574168743232
distane is 55.7326496312656
distane is 59.83891991072511
distane is 26.200885340659497
distane is 20.01278607364791
distane is 45.77507737396205
distane is 23.596062558499476
distane is 30.3055331892702
distane is 21.762097407757363
distane is 29.670295844159746
distane is 18.655417319374685
distane is 28.627699967312516
distane is 22.052215942718586
distane is 93.57959054932829
distane is 50.50285459764855
distane is 28.26447280974603
distane is 23.96323280210695
distane is 17.424663738133816
distane is 18.982021829958814
distane is 20.055562183146293
distane is 21.1247228256314
distane is 20.61018775031514
distane is 14.471994682406155
distane is 16.5079776337471
distane is 18.241192972546095
distane is 16.02539942212222
distane is 20.674451894554423
distane is 22.333042919598647
distane is 20.966295489080345
distane is 25.303083502206675
distane is 20.830395699932673
distane is 15.221446715358091

127.0.0.1 - - [01/Jun/2021 10:47:47] "POST /predict HTTP/1.1" 200 -


distane is 204.66197453547176
distane is 39.99990643221327
distane is 39.131637598200655
distane is 45.6526206957025
distane is 173.12283650019586
distane is 43.356736231264684
distane is 181.3137041516802
distane is 45.28493420422497
distane is 190.80426227945435
distane is 44.83642063769379
distane is 190.08952139532448
distane is 37.549947667455854
distane is 41.9735797704608
distane is 50.6504112117114
distane is 94.28533032379326
distane is 48.81480837632794
distane is 144.1025580435171
distane is 54.427740190181225
distane is 136.44343527991438
distane is 43.791935754638565
distane is 97.26373827491278
distane is 45.081559974491746
distane is 73.09706175064957
distane is 52.05545432827168
distane is 107.57570235018878
distane is 53.01396968413621
distane is 54.517084150803015
distane is 46.07696229535544
distane is 83.66454972529178
distane is 59.51834192692539
distane is 54.51756882329779
distane is 42.50833786908187
distane is 49.637759770372405
distane is 54.72487803800344
dis